In [2]:
import cv2
import pygame
import mediapipe as mp
import numpy as np
import json
import time
from PIL import Image, ImageSequence
from queue import Queue
from threading import Thread
from moviepy import VideoFileClip


# Inicializa los módulos de Mediapipe
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Configuración del video de referencia
dance_video = 'dance_video.mp4'
audio_output_path = "audio.mp3"

play_button = cv2.imread("./assets/play.png", cv2.IMREAD_UNCHANGED)
logo = cv2.imread("./assets/logo.png", cv2.IMREAD_UNCHANGED)
score_screen = cv2.imread("./assets/score.png", cv2.IMREAD_UNCHANGED)
skeleton_on= cv2.imread("./assets/skeleton.png", cv2.IMREAD_UNCHANGED)
skeleton_off= cv2.imread("./assets/skeleton_off.png", cv2.IMREAD_UNCHANGED)

# Variable global para controlar el tiempo de espera
last_toggle_time = 0  # Almacena el tiempo del último toggle
TOGGLE_DELAY = 1  # Tiempo de espera en segundos

# Configuración
process_every_n_frames = 30
toggle_squeleton=  False

# Bandera para detener los hilos
stop_threads = False  

#puntuación
score= 8000

best_score= 0 

# Crear un índice para cambiar entre los frames del GIF
gif_index = 0
# Convertir el GIF en una lista de frames de imágenes
gif_frames = []

#0 pantalla inicio 1 juego 2 puntuación
game_mode= 0



def overlay_image(frame, overlay, x, y, resizex=None, resizey=None):
    """
    Superpone una imagen sobre un frame en las coordenadas (x, y).
    """

    # Redimensionar el overlay si es necesario
    if resizex and resizey:
        overlay = cv2.resize(overlay, (resizex, resizey))
    
    overlay_height, overlay_width = overlay.shape[:2]

    # Verificar los límites del frame
    frame_height, frame_width = frame.shape[:2]
    if y + overlay_height > frame_height or x + overlay_width > frame_width:
        raise ValueError("El overlay excede los límites del frame.")

    # Manejar el canal alfa si está presente
    alpha_overlay = None
    if overlay.shape[2] == 4:
        alpha_overlay = overlay[:, :, 3] / 255.0  # Normaliza el canal alfa
        overlay = overlay[:, :, :3]  # Elimina el canal alfa para la mezcla

    # Seleccionar la región de interés (ROI) en el frame
    roi = frame[y:y+overlay_height, x:x+overlay_width]

    # Mezcla usando el canal alfa (si existe)
    if alpha_overlay is not None:
        for c in range(3):  # Mezcla cada canal (B, G, R)
            roi[:, :, c] = (alpha_overlay * overlay[:, :, c] +
                            (1 - alpha_overlay) * roi[:, :, c])
    else:
        # Sin canal alfa, simplemente reemplaza los píxeles
        frame[y:y+overlay_height, x:x+overlay_width] = overlay

# Función para actualizar el índice del GIF y obtener el siguiente frame
def get_next_gif_frame():
    global gif_index
    if gif_index < len(gif_frames):  # Aumenta el índice solo si hay fotogramas disponibles
        frame = cv2.resize(gif_frames[gif_index], (100, 100))
        gif_index += 1
        return frame
    return None  # Devuelve None después de completar el GIF


def change_gif(score):
    global gif_index
    global gif
    global gif_frames
    
    gif_index=0
    # Cargar el archivo GIF usando PIL
    if score==100:
        gif_path = "./assets/perfect.gif"
    elif score==50:
        gif_path = "./assets/good.gif"
    else:
        gif_path = "./assets/fail.gif"  # Ruta al archivo GIF

    gif = Image.open(gif_path)
    # Convertir el GIF en una lista de frames de imágenes
    gif_frames = []
    for frame in ImageSequence.Iterator(gif):
        frame = frame.convert("RGBA")  # Asegurarse de que el GIF tenga un canal alfa
        gif_frames.append(np.array(frame))
        

def calcular_distancia_escala(puntos):
    hombro_izquierdo = puntos[11]
    hombro_derecho = puntos[12]
    return np.linalg.norm(np.array([hombro_izquierdo["x"], hombro_izquierdo["y"]]) -
                          np.array([hombro_derecho["x"], hombro_derecho["y"]]))

def calcular_similitud(puntos1, puntos2, escala1, escala2):
    if not puntos1 or not puntos2 or len(puntos1) != len(puntos2):
        return 0

    distancias = []
    for p1, p2 in zip(puntos1, puntos2):
        p1_escalado = np.array([p1["x"] / escala1, p1["y"] / escala1, p1["z"] / escala1])
        p2_escalado = np.array([p2["x"] / escala2, p2["y"] / escala2, p2["z"] / escala2])
        distancias.append(np.linalg.norm(p1_escalado - p2_escalado))

    return np.mean(distancias)

def obtener_puntos_por_frame(json_data, frame_buscado):
    for entrada in json_data:
        if entrada["frame"] == frame_buscado:
            return entrada["puntos"]
    return None

def puntuacion(json_file, frame_buscado, results_cam):
    puntos_referencia = obtener_puntos_por_frame(json_file, frame_buscado)
    if puntos_referencia:
        puntos_usuario = [
            {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
            for lm in results_cam.pose_landmarks.landmark
        ]

        escala_referencia = calcular_distancia_escala(puntos_referencia)
        escala_usuario = calcular_distancia_escala(puntos_usuario)

        similitud = calcular_similitud(puntos_referencia, puntos_usuario, escala_referencia, escala_usuario)

        if similitud < 5:
            change_gif(100)
            return 100
        elif similitud < 10:
            change_gif(50)
            return 50
        else:
            change_gif(0)
            return 0
    return 0


start_music = True
def process_video(cap_video, video_queue, video_frame_rate, frame_width, frame_height, puntos_video_referencia):
    global stop_threads, game_mode, start_music, score, best_score

    linea= 0
    
    while cap_video.isOpened() and not stop_threads:
        if game_mode == 1:
            linea= 0
            if start_music:
                pygame.mixer.music.play()
                cap_video.set(cv2.CAP_PROP_POS_FRAMES, 0)
                start_music = False
            current_time = pygame.mixer.music.get_pos() / 1_000
            current_frame_index = int(current_time * video_frame_rate)

            cap_video.set(cv2.CAP_PROP_POS_FRAMES, current_frame_index)
            ret_video, frame_video = cap_video.read()
            if not ret_video:
                game_mode = 2
                continue

            frame_rgb = cv2.cvtColor(frame_video, cv2.COLOR_BGR2RGB)
            frame_rgb = cv2.resize(frame_rgb, (frame_width, frame_height))  # Reducir tamaño

            gif_frame = get_next_gif_frame()
            if gif_frame is not None:
                overlay_image(frame_rgb, gif_frame, 0, 0)
            
            if toggle_squeleton==True:
                puntos_referencia = obtener_puntos_por_frame(puntos_video_referencia, current_frame_index)
                # Dibujar puntos y conexiones en el video de referencia
                if puntos_referencia:
                    for conexion in mp_pose.POSE_CONNECTIONS:
                        inicio = puntos_referencia[conexion[0]]
                        fin = puntos_referencia[conexion[1]]
                        inicio_px = (int(inicio["x"] * frame_width), int(inicio["y"] * frame_height))
                        fin_px = (int(fin["x"] * frame_width), int(fin["y"] * frame_height))
                        cv2.line(frame_rgb, inicio_px, fin_px, (255, 0, 0), 2)

                    for punto in puntos_referencia:
                        px = (int(punto["x"] * frame_width), int(punto["y"] * frame_height))
                        cv2.circle(frame_rgb, px, 5, (0, 255, 0), -1)

        elif game_mode == 0:
            frame_rgb = cv2.cvtColor(logo, cv2.COLOR_BGR2RGB)
            frame_rgb = cv2.resize(frame_rgb, (frame_width, frame_height)) 
        elif game_mode == 2:

            if best_score < score:
                best_score = score
            frame_rgb = cv2.cvtColor(score_screen, cv2.COLOR_BGR2RGB)
            frame_rgb = cv2.resize(frame_rgb, (frame_width, frame_height)) 
            cv2.putText(frame_rgb, f'Puntuacion: {best_score}', (110, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame_rgb, f'Mejor puntuacion: {score}', (110, 125),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            if linea < 440*score/20000 and linea < 439:
                linea+=1
            cv2.rectangle(frame_rgb,(110, 140),(550 ,170),(0,0,0),-1)
            cv2.rectangle(frame_rgb,(111, 141),(110 + linea ,169),(255,0,0),-1)

        video_queue.put(frame_rgb)

    cap_video.release()


def process_camera(cap_camera, camera_queue, puntos_video_referencia, video_frame_rate, frame_width, frame_height, cap_video):
    global stop_threads, game_mode, score, toggle_squeleton, last_toggle_time, TOGGLE_DELAY, start_music

    # Coordenadas del botón (ajusta según sea necesario)
    button_x, button_y = 50, 100
    button_width, button_height = 80, 80

    # Coordenadas del esqueleto (ajusta según sea necesario)
    skeleton_x, skeleton_y = 50, 200
    skeleton_width, skeleton_height = 80, 120
    
    pose = None
    hands = None
    
    while cap_camera.isOpened() and not stop_threads:
        ret_camera, frame_camera = cap_camera.read()

        if not ret_camera:
            break

        # Initialize the frame that will be processed
        frame_camera = cv2.flip(frame_camera, 1)
        frame_camera = cv2.resize(frame_camera, (frame_width, frame_height))
        rgb_frame_cam = cv2.cvtColor(frame_camera, cv2.COLOR_BGR2RGB)

        if game_mode == 1:
            # Initialize Pose model if it's not already initialized
            if pose is None:
                pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

            current_time = pygame.mixer.music.get_pos() / 1_000
            current_frame_index = int(current_time * video_frame_rate)

            if toggle_squeleton == True:
                results_cam = pose.process(rgb_frame_cam)
                if results_cam.pose_landmarks:

                    mp_drawing.draw_landmarks(
                        rgb_frame_cam, results_cam.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                        mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2)
                    )
                    if current_frame_index % process_every_n_frames == 0:
                        score += puntuacion(puntos_video_referencia, current_frame_index, results_cam)


            elif current_frame_index % process_every_n_frames == 0:
                # Process with Pose model
                results_cam = pose.process(rgb_frame_cam)
                print(current_frame_index)
                if results_cam.pose_landmarks:
                    score += puntuacion(puntos_video_referencia, current_frame_index, results_cam)

                
            cv2.putText(rgb_frame_cam, f'Puntuacion: {score}', (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
        else:
            # Initialize Hands model if it's not already initialized
            if hands is None:
                hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)
                
            # Process with Hands model
            results_cam = hands.process(rgb_frame_cam)

            # Dibujar el botón de "play"
            overlay_image(rgb_frame_cam, play_button, button_x, button_y, button_width, button_height)

            if toggle_squeleton:
                overlay_image(rgb_frame_cam, skeleton_on, skeleton_x, skeleton_y, skeleton_width, skeleton_height)
            else:
                overlay_image(rgb_frame_cam, skeleton_off, skeleton_x, skeleton_y, skeleton_width, skeleton_height)

        
            if results_cam.multi_hand_landmarks:
                for hand_landmarks in results_cam.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        rgb_frame_cam, hand_landmarks, mp_hands.HAND_CONNECTIONS,  # Use HAND_CONNECTIONS instead of POSE_CONNECTIONS
                        mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                        mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2)
                    )
                
                # Check if the hand is inside the play button area
                for landmark in hand_landmarks.landmark:
                    hand_x = int(landmark.x * frame_width)
                    hand_y = int(landmark.y * frame_height)

                    if button_x <= hand_x <= button_x + button_width and button_y <= hand_y <= button_y + button_height:
                        game_mode = 1  # Switch to game mode
                        score = 0
                        start_music = True
                        pose = None  # Reset the Pose model when switching to game mode
                    if skeleton_x <= hand_x <= skeleton_x + skeleton_width and skeleton_y <= hand_y <= skeleton_y + skeleton_height:
                        current_time = time.time()  # Obtiene el tiempo actual
                        if current_time - last_toggle_time > TOGGLE_DELAY:  # Verifica si ha pasado suficiente tiempo
                            toggle_squeleton = not toggle_squeleton  # Cambia el modo de juego
                            last_toggle_time = current_time  # Actualiza el tiempo del último toggle

        # Put the processed frame into the queue
        camera_queue.put(rgb_frame_cam)


    



def jugar_con_puntos(json_file, video_path):
    global stop_threads

    with open(json_file, 'r') as f:
        puntos_video_referencia = json.load(f)

    pygame.init()
    pygame.mixer.music.load(audio_output_path)

    cap_video = cv2.VideoCapture(video_path)
    cap_camera = cv2.VideoCapture(0)

    if not cap_video.isOpened() or not cap_camera.isOpened():
        print("Error: No se pudo abrir el video o la cámara.")
        return

    frame_width = 640  # Reducir a la mitad
    frame_height = 360  # Reducir a la mitad
    video_frame_rate = cap_video.get(cv2.CAP_PROP_FPS)

    screen_width = frame_width * 2  # Doblar para mostrar video + cámara
    screen_height = frame_height
    screen = pygame.display.set_mode((screen_width, screen_height))  # Usar dimensiones reducidas

    video_queue = Queue()
    camera_queue = Queue()

    video_thread = Thread(target=process_video, args=(cap_video, video_queue, video_frame_rate, frame_width, frame_height, puntos_video_referencia))
    camera_thread = Thread(target=process_camera, args=(cap_camera, camera_queue, puntos_video_referencia, video_frame_rate, frame_width, frame_height, cap_video))

    video_thread.start()
    camera_thread.start()

    while video_thread.is_alive() or camera_thread.is_alive():
        if not video_queue.empty():
            frame_rgb = video_queue.get()
            current_image = pygame.image.frombuffer(frame_rgb.tobytes(), frame_rgb.shape[1::-1], "RGB")
            screen.blit(current_image, (0, 0))

        if not camera_queue.empty():
            frame_camera_rgb = camera_queue.get()
            frame_camera_surface = pygame.image.frombuffer(frame_camera_rgb.tobytes(), frame_camera_rgb.shape[1::-1], "RGB")
            screen.blit(frame_camera_surface, (frame_width, 0))

        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                stop_threads = True
                video_thread.join()
                camera_thread.join()
                cap_video.release()
                cap_camera.release()
                pygame.quit()
                return

    stop_threads = True
    video_thread.join()
    camera_thread.join()
    cap_video.release()
    cap_camera.release()
    pygame.quit()

# Función para extraer puntos de referencia y guardarlos en un archivo JSON
def extraer_y_guardar_puntos(video_path, output_file):
    puntos_video_referencia = []
    # Cargar el video y extraer el audio
    video_clip = VideoFileClip(video_path)
    video_clip.audio.write_audiofile(audio_output_path)

    print("Audio extraído y guardado en:", audio_output_path)
    print("Se va a procesar el video, este proceso puede durar varios minutos...")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        video_cap = cv2.VideoCapture(video_path)
        while True:
            ret_vid, frame_vid = video_cap.read()
            if not ret_vid:
                break

            # Obtén el índice del frame actual
            frame_index = int(video_cap.get(cv2.CAP_PROP_POS_FRAMES))

            rgb_frame_vid = cv2.cvtColor(frame_vid, cv2.COLOR_BGR2RGB)
            results_vid = pose.process(rgb_frame_vid)

            if results_vid.pose_landmarks:
                puntos = []
                for landmark in results_vid.pose_landmarks.landmark:
                    puntos.append({
                        "x": landmark.x,
                        "y": landmark.y,
                        "z": landmark.z,
                        "visibility": landmark.visibility
                    })

                # Agrega los puntos junto con el índice del frame al resultado
                puntos_video_referencia.append({
                    "frame": frame_index,
                    "puntos": puntos
                })

        video_cap.release()

    # Guarda los puntos en un archivo JSON
    with open(output_file, 'w') as f:
        json.dump(puntos_video_referencia, f)

    print(f"Puntos de referencia guardados en {output_file}")

if __name__ == "__main__":
    modo = input("Seleccione modo: 'extraer' para guardar puntos o 'jugar' para comparar: ")

    if modo == 'extraer':
        extraer_y_guardar_puntos(dance_video, 'puntos_referencia.json')
    elif modo == 'jugar':
        jugar_con_puntos('puntos_referencia.json', dance_video)

